In [32]:
import pyodbc
server = 'DIST-6-505.uopnet.plymouth.ac.uk'
database = 'COMP2001_MFerguson'
username = 'MFerguson'
password = 'GjiF140*'
driver = '{ODBC Driver 17 for SQL Server}'

conn_str = (
    f'DRIVER={driver};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    'Encrypt=Yes;'
    'TrustServerCertificate=Yes;'
    'Connection Timeout=30;'
    'Trusted_Connection=No;'
)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [ ]:
cursor.execute(''' CREATE SCHEMA CW2''')

# Creating the tables

In [ ]:
cursor.execute('''
CREATE TABLE CW2.[User] (
    user_id CHAR(8) PRIMARY KEY,
    username VARCHAR(255) NOT NULL,
    email VARCHAR(255) NOT NULL,
    [password] VARCHAR(255) NOT NULL,
    role VARCHAR(8) NOT NULL,
    
    CHECK (user_id LIKE '[U][S][R][0-9][0-9][0-9][0-9][0-9]'),
    CHECK (LEN(username) >= 5 AND LEN(username) <= 255),
    CHECK (email LIKE '%_@_%.__%'),
    CHECK (LEN([password]) >= 8 AND CHARINDEX(' ', [password]) = 0),
    CHECK (role = 'admin' OR role = 'user')
)
''')
conn.commit()

In [ ]:
cursor.execute('''
INSERT INTO CW2.[User] (user_id, username, email, [password], role)
VALUES
('USR00001','Admin','admin@gmail.com','AdM1NpQsW0Rd','admin'),
('USR00002','Tim Berners-Lee','tim@plymouth.ac.uk','COMP2001!', 'user'),
('USR00003','Grace Hopper','grace@plymouth.ac.uk','ISAD123!', 'user'),
('USR00004','Ada Lovelace','ada@plymouth.ac.uk','insecurePassword','user');
''')
conn.commit()

In [ ]:
cursor.execute('''
CREATE TABLE CW2.Location_Point (
    location_point_id CHAR(8) PRIMARY KEY,
    latitude DECIMAL(9,6) NOT NULL,
    longitude DECIMAL(9,6) NOT NULL,
    description VARCHAR(255),      
    CHECK (location_point_id LIKE '[L][P][T][0-9][0-9][0-9][0-9][0-9]'),
    CHECK (latitude BETWEEN -90 AND 90),
    CHECK (longitude BETWEEN -180 AND 180),    
    CHECK (LEN(description) >3)       
)
''')
conn.commit()

In [ ]:
cursor.execute('''
INSERT INTO CW2.Location_Point (location_point_id, latitude, longitude, description)
VALUES
('LPT00001', 50.123456, -4.234567, 'Scenic viewpoint on the hill'),
('LPT00002', 51.654321, -3.876543, 'Lake surrounded by forest'),
('LPT00003', 52.987654, -2.123456, 'Historic site with ruins'),
('LPT00004', 53.345678, -1.654321, 'Trailhead with parking area');
''')
conn.commit()


In [24]:
cursor.execute('''
CREATE TABLE CW2.Trail (
    trail_id CHAR(8) PRIMARY KEY, 
    trail_name VARCHAR(255) NOT NULL UNIQUE,
    summary VARCHAR(255),
    description VARCHAR(1500),
    location VARCHAR(255) NOT NULL,
    traffic VARCHAR(8) NOT NULL,
    difficulty VARCHAR(8) NOT NULL,
    length DECIMAL(7,2) NOT NULL,           
    duration VARCHAR(5) NOT NULL,
    elevation_gain INT NOT NULL,
    route_type VARCHAR(14) NOT NULL,
               
    owner_id CHAR(8) NOT NULL, 
    location_pt_1 CHAR(8),
    location_pt_2 CHAR(8),
    location_pt_3 CHAR(8),
    location_pt_4 CHAR(8),
    location_pt_5 CHAR(8),
                   
    CHECK (trail_id LIKE '[T][R][L][0-9][0-9][0-9][0-9][0-9]'), 
    CHECK (LEN(trail_name) >= 5 AND LEN(trail_name) <= 50),
    CHECK (LEN(summary) >= 10 OR LEN(summary) = 0),
    CHECK (LEN(description) >= 50 OR LEN(description) = 0),
    CHECK (lower(traffic)= 'heavy' OR lower(traffic) = 'moderate' OR lower(traffic) = 'light'),
    CHECK (lower(difficulty) = 'hard' OR lower(difficulty) = 'moderate' OR lower(difficulty) = 'easy'),  
    CHECK (length > 0 AND length = ROUND(length, 2)),
    CHECK (duration LIKE'[0-9][0-9]:[0-9][0-9]'),
    CHECK (elevation_gain >=0),
    CHECK (lower(route_type) = 'out & back' OR lower(route_type) = 'loop' OR
    lower(route_type) = 'point to point'),
               
    CONSTRAINT FK_owner_id FOREIGN KEY (owner_id) REFERENCES CW2.[User](user_id),
    CONSTRAINT fk_location_pt_1 FOREIGN KEY (location_pt_1) REFERENCES CW2.Location_Point(location_point_id),
    CONSTRAINT fk_location_pt_2 FOREIGN KEY (location_pt_2) REFERENCES CW2.Location_Point(location_point_id),
    CONSTRAINT fk_location_pt_3 FOREIGN KEY (location_pt_3) REFERENCES CW2.Location_Point(location_point_id),
    CONSTRAINT fk_location_pt_4 FOREIGN KEY (location_pt_4) REFERENCES CW2.Location_Point(location_point_id),
    CONSTRAINT fk_location_pt_5 FOREIGN KEY (location_pt_5) REFERENCES CW2.Location_Point(location_point_id)
)
''')
conn.commit()


In [25]:
cursor.execute('''
INSERT INTO CW2.Trail (
    trail_id, trail_name, summary, description, location, traffic,
    difficulty, length, duration, elevation_gain, route_type, owner_id,
    location_pt_1, location_pt_2, location_pt_3, location_pt_4, location_pt_5
)
VALUES
('TRL00001', 'Forest Walk', 'A scenic walk through dense forest',
 'This trail takes you through a beautiful forest with diverse wildlife and flora.', 'Plymouth, Devon, UK','moderate',
 'easy', 5.50, '02:30', 300, 'loop', 'USR00002',
 'LPT00001', 'LPT00002', 'LPT00003', 'LPT00004', 'LPT00001'),
('TRL00002','Mountain Hike', 'A challenging hike with rewarding views',
 'A strenuous but rewarding trail up the mountain with breathtaking views at the summit.', 'Los Angeles, California, USA', 'heavy', 
 'hard', 12.30, '05:45', 1200, 'out & back', 'USR00003',
 'LPT00002', 'LPT00004', 'LPT00001', 'LPT00002', 'LPT00003'),
('TRL00003', 'Coastal Path', 'A leisurely walk along the coast', 
 'This trail offers stunning views of the ocean and a chance to visit sandy beaches.', 'Lisbon, Portugal', 'light',
 'moderate', 8.70, '03:15', 150, 'point to point', 'USR00004',
 'LPT00003', 'LPT00004', 'LPT00001', 'LPT00002', 'LPT00003'),
('TRL00004', 'River Valley Trek', 'A trek along a scenic river valley',
 'Follow the river through a picturesque valley with plenty of photo opportunities.', 'Moscow, Russia','moderate',
 'easy', 10.20, '04:00', 400, 'loop', 'USR00002',
 'LPT00004', 'LPT00001', 'LPT00002', 'LPT00003', 'LPT00004');

''')
conn.commit()

In [26]:
cursor.execute('''
CREATE TABLE CW2.Attraction (
    attraction_id CHAR(8) PRIMARY KEY,
    attraction_name VARCHAR(255) NOT NULL,

    CHECK (attraction_id LIKE '[A][T][T][0-9][0-9][0-9][0-9][0-9]'),
    CHECK (LEN(attraction_name) >= 3)               
)
''')
conn.commit()

In [27]:
cursor.execute('''
INSERT INTO CW2.Attraction (attraction_id, attraction_name)
VALUES
('ATT00001', 'Waterfall'),
('ATT00002', 'Viewpoint'),
('ATT00003', 'Historic Ruins'),
('ATT00004', 'Picnic Spot');
''')
conn.commit()


In [28]:
cursor.execute('''
CREATE TABLE CW2.Trail_Attraction (
    trail_id CHAR(8) NOT NULL,
    attraction_id CHAR(8) NOT NULL,
    PRIMARY KEY (trail_id, attraction_id),
    
    CONSTRAINT FK_trail_id_attraction FOREIGN KEY (trail_id) REFERENCES CW2.Trail(trail_id),
    CONSTRAINT FK_attraction_id FOREIGN KEY (attraction_id) REFERENCES CW2.Attraction(attraction_id)
)
''')
conn.commit()

In [29]:
cursor.execute('''
INSERT INTO CW2.Trail_Attraction (trail_id, attraction_id)
VALUES
('TRL00001', 'ATT00001'),
('TRL00002', 'ATT00002'),
('TRL00003', 'ATT00003'),
('TRL00004', 'ATT00004'),
('TRL00001', 'ATT00002'),
('TRL00002', 'ATT00004'),
('TRL00003', 'ATT00001');
''')
conn.commit()



# Drop all tables

In [ ]:
cursor.execute('''
DROP TABLE CW2.Trail_Attraction
DROP TABLE CW2.Trail
DROP TABLE CW2.Location_Point
DROP TABLE CW2.Attraction
DROP TABLE CW2.[User]
''')
conn.commit()

# A view to trail and attractions

In [ ]:
cursor.execute('''

CREATE VIEW CW2.TrailView AS
SELECT 
    Trail.trail_id,
    Trail.trail_name,
    Trail.summary,
    Trail.description,
    Trail.location,
    Trail.traffic,
    Trail.difficulty,
    Trail.length,
    Trail.duration,
    Trail.elevation_gain,
    Trail.route_type,
    Trail.owner_id,
    STRING_AGG(
        CONCAT(
            'Latitude: ', Location_Point.latitude, 
            ', Longitude: ', Location_Point.longitude
        ), '; '
    ) AS location_points,
    STRING_AGG(Attraction.attraction_name, ', ') AS attractions
FROM 
    CW2.Trail
LEFT JOIN 
    CW2.Location_Point ON Location_Point.location_point_id IN (
        Trail.location_pt_1, 
        Trail.location_pt_2, 
        Trail.location_pt_3, 
        Trail.location_pt_4, 
        Trail.location_pt_5
    )
LEFT JOIN 
    CW2.Trail_Attraction ON Trail.trail_id = Trail_Attraction.trail_id
LEFT JOIN 
    CW2.Attraction ON Trail_Attraction.attraction_id = Attraction.attraction_id
GROUP BY 
    Trail.trail_id,
    Trail.trail_name,
    Trail.summary,
    Trail.description,
    Trail.location,
    Trail.traffic,
    Trail.difficulty,
    Trail.length,
    Trail.duration,
    Trail.elevation_gain,
    Trail.route_type,
    Trail.owner_id;

''')
conn.commit()


# Stored Procedures for CRUD for each table

In [33]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateTrail (@trail_id as CHAR(8), @owner_id as CHAR(8), @trail_name as VARCHAR(255), @summary as VARCHAR(255),  
@description as VARCHAR(1500), @location as VARCHAR(255), @traffic as VARCHAR(8), @difficulty as VARCHAR(8), 
@length as DECIMAL(7,2), @duration as VARCHAR(5), @elevation_gain as INT, @route_type as VARCHAR(14), 
@location_pt_1 as CHAR(8), @location_pt_2 as CHAR(8), @location_pt_3 as CHAR(8), @location_pt_4 as CHAR(8), @location_pt_5 as CHAR(8))
AS
BEGIN   
    INSERT INTO CW2.Trail (trail_id, owner_id, trail_name, summary, description, location, traffic, difficulty, length, duration, 
    elevation_gain, route_type, location_pt_1, location_pt_2, location_pt_3, location_pt_4, location_pt_5)
    VALUES 
    (@trail_id, @owner_id, @trail_name, @summary, @description, @location, @traffic, @difficulty, @length, @duration,
    @elevation_gain, @route_type, @location_pt_1, @location_pt_2, @location_pt_3, @location_pt_4, @location_pt_5);
END;
''')
conn.commit()

In [34]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveTrail (@trail_id CHAR(8) = NULL) AS
BEGIN
    IF @trail_id IS NULL
    BEGIN
        SELECT * FROM CW2.TrailView;
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.TrailView
        WHERE trail_id = @trail_id;
    END
END
''')
conn.commit()

In [ ]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveTrailByOwner (@owner_id CHAR(8)) AS
BEGIN
    SELECT * FROM CW2.TrailView
    WHERE owner_id = @owner_id;
END
''')
conn.commit()

In [36]:
cursor.execute('''
Create PROCEDURE CW2.RetrieveTrailByLocation (@location as VARCHAR(255)) AS
BEGIN
    SELECT * FROM CW2.TrailView
    WHERE location = @location;
END
''')
conn.commit()

In [ ]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveTrailByAttraction (@attraction_id CHAR(8)) AS
BEGIN
    SELECT * FROM CW2.TrailView
    WHERE trail_id IN (
        SELECT trail_id FROM CW2.Trail_Attraction
        WHERE attraction_id = @attraction_id
    )
END
''')
conn.commit()

In [ ]:
cursor.execute('''
CREATE PROCEDURE CW2.UpdateTrail (@trail_id as CHAR(8) = NULL, @owner_id as CHAR(8)= NULL, @trail_name as VARCHAR(255) = NULL, @summary as VARCHAR(255) = NULL,  
@description as VARCHAR(1500) = NULL, @location as VARCHAR(255) = NULL, @traffic as VARCHAR(8) = NULL, @difficulty as VARCHAR(8) = NULL, 
@length as DECIMAL(7,2) = NULL, @duration as VARCHAR(5) = NULL, @elevation_gain as INT = NULL, @route_type as VARCHAR(14) = NULL, 
@location_pt_1 as CHAR(8)= NULL, @location_pt_2 as CHAR(8) = NULL, @location_pt_3 as CHAR(8) = NULL, @location_pt_4 as CHAR(8) = NULL, @location_pt_5 as CHAR(8) = NULL)
AS
BEGIN
    BEGIN TRANSACTION;

    BEGIN TRY
        IF NOT EXISTS (SELECT 1 FROM CW2.Trail WHERE trail_id = @trail_id)
        BEGIN
            RAISERROR('Trail ID does not exist.', 16, 1);
            ROLLBACK TRANSACTION;
            RETURN;
        END;
        UPDATE CW2.Trail
        SET
            owner_id = ISNULL(@owner_id, owner_id),
            trail_name = ISNULL(@trail_name, trail_name),
            summary = ISNULL(@summary, summary),
            description = ISNULL(@description, description),
            location = ISNULL(@location, location),
            traffic = ISNULL(@traffic, traffic),
            difficulty = ISNULL(@difficulty, difficulty),
            length = ISNULL(@length, length),
            duration = ISNULL(@duration, duration),
            elevation_gain = ISNULL(@elevation_gain, elevation_gain),
            route_type = ISNULL(@route_type, route_type),
            location_pt_1 = ISNULL(@location_pt_1, location_pt_1),
            location_pt_2 = ISNULL(@location_pt_2, location_pt_2),
            location_pt_3 = ISNULL(@location_pt_3, location_pt_3),
            location_pt_4 = ISNULL(@location_pt_4, location_pt_4),
            location_pt_5 = ISNULL(@location_pt_5, location_pt_5)
        WHERE trail_id = @trail_id;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;
''')
conn.commit()

In [39]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteTrail(@trail_id as CHAR(8)) AS
BEGIN
    DELETE FROM CW2.Trail_Attraction
    WHERE trail_id = @trail_id;
    DELETE FROM CW2.Trail
    WHERE trail_id = @trail_id
END
''')
conn.commit()

In [40]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateUser (@user_id as CHAR(8), @username as VARCHAR(255), @email as VARCHAR(255), @password as VARCHAR(255), @role as VARCHAR(8))
AS
BEGIN
    INSERT INTO CW2.[User] (user_id, username, email, [password], role)
    VALUES (@user_id, @username, @email, @password, @role);
END
''')
conn.commit()

In [41]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveUser (@user_id CHAR(8) = NULL) AS
BEGIN
    IF @user_id IS NULL
    BEGIN
        SELECT * FROM CW2.[User];
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.[User]
        WHERE user_id = @user_id;
    END
END
''')
conn.commit()

In [42]:
cursor.execute('''
CREATE PROCEDURE CW2.UpdateUser (@user_id as CHAR(8) = NULL, @username as VARCHAR(255) = NULL, @email as VARCHAR(255) = NULL, @password as VARCHAR(255) = NULL, @role as VARCHAR(8) = NULL)
AS
BEGIN
    BEGIN TRANSACTION;

    BEGIN TRY
        IF NOT EXISTS (SELECT 1 FROM CW2.[User] WHERE user_id = @user_id)
        BEGIN
            RAISERROR('User ID does not exist.', 16, 1);
            ROLLBACK TRANSACTION;
            RETURN;
        END;
        UPDATE CW2.[User]
        SET
            username = ISNULL(@username, username),
            email = ISNULL(@email, email),
            [password] = ISNULL(@password, [password]),
            role = ISNULL(@role, role)
        WHERE user_id = @user_id;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;
''')
conn.commit()

In [43]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteUser(@user_id as CHAR(8)) AS
BEGIN
    DELETE FROM CW2.Trail
    WHERE owner_id = @user_id
    DELETE FROM CW2.[User]
    WHERE user_id = @user_id
END
''')
conn.commit()

In [44]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateLocationPoint (@location_point_id as CHAR(8), @latitude as DECIMAL(9,6), @longitude as DECIMAL(9,6), @description as VARCHAR(255))
AS
BEGIN
    INSERT INTO CW2.Location_Point (location_point_id, latitude, longitude, description)
    VALUES (@location_point_id, @latitude, @longitude, @description);
END
''')
conn.commit()

In [45]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveLocationPoint (@location_point_id CHAR(8) = NULL) AS
BEGIN
    IF @location_point_id IS NULL
    BEGIN
        SELECT * FROM CW2.Location_Point;
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.Location_Point
        WHERE location_point_id = @location_point_id;
    END
END
''')
conn.commit()

In [46]:
cursor.execute('''
CREATE PROCEDURE CW2.UpdateLocationPoint (@location_point_id as CHAR(8) = NULL, @latitude as DECIMAL(9,6) = NULL, @longitude as DECIMAL(9,6) = NULL, @description as VARCHAR(255) = NULL)
AS
BEGIN
    BEGIN TRANSACTION;

    BEGIN TRY
        IF NOT EXISTS (SELECT 1 FROM CW2.Location_Point WHERE location_point_id = @location_point_id)
        BEGIN
            RAISERROR('Location Point ID does not exist.', 16, 1);
            ROLLBACK TRANSACTION;
            RETURN;
        END;
        UPDATE CW2.Location_Point
        SET
            latitude = ISNULL(@latitude, latitude),
            longitude = ISNULL(@longitude, longitude),
            description = ISNULL(@description, description)
        WHERE location_point_id = @location_point_id;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;
''')
conn.commit()

In [47]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteLocationPoint(@location_point_id as CHAR(8)) AS
BEGIN
    DELETE FROM CW2.Trail
    WHERE location_pt_1 = @location_point_id OR location_pt_2 = @location_point_id OR location_pt_3 = @location_point_id OR location_pt_4 = @location_point_id OR location_pt_5 = @location_point_id
    DELETE FROM CW2.Location_Point
    WHERE location_point_id = @location_point_id
END
''')
conn.commit()

In [48]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateAttraction (@attraction_id as CHAR(8), @attraction_name as VARCHAR(255))
AS
BEGIN
    INSERT INTO CW2.Attraction (attraction_id, attraction_name)
    VALUES (@attraction_id, @attraction_name);
END
''')
conn.commit()

In [49]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveAttraction (@attraction_id CHAR(8) = NULL) AS
BEGIN
    IF @attraction_id IS NULL
    BEGIN
        SELECT * FROM CW2.Attraction;
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.Attraction
        WHERE attraction_id = @attraction_id;
    END
END
''')
conn.commit()

In [ ]:
cursor.execute('''
CREATE PROCEDURE CW2.UpdateAttraction (@attraction_id as CHAR(8) = NULL, @attraction_name as VARCHAR(255) = NULL)
AS
BEGIN
    BEGIN TRANSACTION;

    BEGIN TRY
        IF NOT EXISTS (SELECT 1 FROM CW2.Attraction WHERE attraction_id = @attraction_id)
        BEGIN
            RAISERROR('Attraction ID does not exist.', 16, 1);
            ROLLBACK TRANSACTION;
            RETURN;
        END;
        UPDATE CW2.Attraction
        SET
            attraction_name = ISNULL(@attraction_name, attraction_name)
        WHERE attraction_id = @attraction_id;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;
''')
conn.commit()

In [51]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteAttraction(@attraction_id as CHAR(8)) AS
BEGIN
    DELETE FROM CW2.Trail_Attraction
    WHERE attraction_id = @attraction_id
    DELETE FROM CW2.Attraction
    WHERE attraction_id = @attraction_id
END
''')
conn.commit()

In [53]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateTrailAttraction (@trail_id as CHAR(8), @attraction_id as CHAR(8))
AS
BEGIN
    INSERT INTO CW2.Trail_Attraction (trail_id, attraction_id)
    VALUES (@trail_id, @attraction_id);
END
''')
conn.commit()

In [54]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveTrailAttraction (@trail_id CHAR(8) = NULL, @attraction_id CHAR(8) = NULL) AS
BEGIN
    IF @trail_id IS NULL AND @attraction_id IS NULL
    BEGIN
        SELECT * FROM CW2.Trail_Attraction;
    END
    ELSE IF @trail_id IS NOT NULL AND @attraction_id IS NULL
    BEGIN
        SELECT * FROM CW2.Trail_Attraction
        WHERE trail_id = @trail_id;
    END
    ELSE IF @trail_id IS NULL AND @attraction_id IS NOT NULL
    BEGIN
        SELECT * FROM CW2.Trail_Attraction
        WHERE attraction_id = @attraction_id;
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.Trail_Attraction
        WHERE trail_id = @trail_id AND attraction_id = @attraction_id;
    END
END
''')
conn.commit()

In [55]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteTrailAttraction(@trail_id as CHAR(8), @attraction_id as CHAR(8)) AS
BEGIN
    DELETE FROM CW2.Trail_Attraction
    WHERE trail_id = @trail_id AND attraction_id = @attraction_id
END
''')
conn.commit()